In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import sympy as sym

import symbolics

In [2]:
%matplotlib inline

# Maximizing equilibrium total offspring

In [29]:
x1, x2, x3, x4 = sym.symbols('x1, x2, x3, x4', real=True, nonnegative=True)
T, R, P, S = sym.symbols('T, R, P, S', real=True, positive=True)

UGA = symbolics.UGA
UgA = symbolics.UgA

total_offspring = symbolics.total_offspring(x1, x2, x3, UGA, UgA, T, R, P, S)

## Substitute equilibrium values

In [30]:
UGA_star, UgA_star, xG_star = sym.symbols("UGA_star, UgA_star, xG_star")
equilibrium_total_offspring = total_offspring.subs({UGA(x1+x3): UGA_star, UgA(x1+x3): UgA_star, x1+x2: xG_star})

In [31]:
equilibrium_total_offspring

2*P*(xG_star*(-UGA_star + 1)**2 + (-UgA_star + 1)**2*(-xG_star + 1)) + 2*R*(UGA_star**2*xG_star + UgA_star**2*(-xG_star + 1)) + (2*S + 2*T)*(UGA_star*xG_star*(-UGA_star + 1) + UgA_star*(-UgA_star + 1)*(-xG_star + 1))

## Make an interactive plot

In [16]:
_equilibrium_total_offspring = sym.lambdify((xG_star, UGA_star, UgA_star, T, R, P, S),
                                            equilibrium_total_offspring,
                                            modules="numpy")

In [32]:
def plot_total_offspring(xG_star, T, R, P, S):

    fig, ax = plt.subplots(1, 1, figsize=(20, 10))
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{gA}^*$", fontsize=20)
    ax.set_title(r"Equilibrium total offspring for $x_G^*$={}".format(xG_star), fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100).reshape(-1, 1)
    UGAs = equilibrium_selection_probs.reshape(-1, 1)
    UgAs = equilibrium_selection_probs.reshape(1, -1)
    Z = _equilibrium_total_offspring(xG_star, UGAs, UgAs, T, R, P, S)
    cax = ax.imshow(Z, origin="lower")

    contours = ax.contour(Z, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs, _ = plt.xticks()
    plt.xticks(locs[1:], np.linspace(0, 1, locs.size-1))
    locs, _ = plt.yticks()
    plt.yticks(locs[1:], np.linspace(0, 1, locs.size-1))

    plt.show()

In [33]:
xG_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description=r"$x_G^*$")

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=10, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

w = widgets.interactive(plot_total_offspring, xG_star=xG_slider, T=T_slider, R=R_slider, P=P_slider, S=S_slider)
display(w)

A Jupyter Widget

## Find the optimal values of $x_G^*, U_{GA}^*, U_{gA}^*$.

The number of total offspring can be written as a function of the equilibrium selection probability.

$$ N\big(x_G^*, U_{GA}^*, U_{gA}^*\big) = 2\bigg(\big((R + P) - (T + S)\big)x_G^*U_{GA}^{*2} + \big((T + S) - 2P\big)x_G^*U_{GA}^* + P - \big((R + P) - (T + S)\big)x_G^*U_{gA}^{*2} - \big((T + S) - 2P\big)x_G^*U_{gA}^* + \big((R + P) - (T + S)\big)U_{gA}^{*2} + \big((T + S) - 2P\big)U_{gA}^* \bigg)$$


To find the equilibrium selection probability that maximizes the number of total offspring we need to solve the following constrained optimization problem.

$$ \max_{x_G^*, U_{GA}^*, U_{gA}^*}\ N\big(x_G^*, U_{GA}^*, U_{gA}^*\big)  $$

subject to the following inequality constraints.

\begin{align}
    -x_G^* \le& 0 \\
    x_G^* - 1 \le& 0 \\
    -U_{GA}^* \le& 0 \\
    U_{GA}^* - 1 \le& 0 \\
    -U_{gA}^* \le& 0 \\
    U_{gA}^* - 1 \le& 0
\end{align}

First-order conditions are as follows.
\begin{align}
    2\bigg(\big((R + P) - (T + S)\big)U_{GA}^{*2} + \big((T + S) - 2P\big)U_{GA}^* + P - \big((R + P) - (T + S)\big)U_{gA}^{*2} - \big((T + S) - 2P\big)U_{gA}^*\bigg) =& -\mu_{x_G^*, 0} + \mu_{x_G^*,1} \\
    2\bigg(2\big((R + P) - (T + S)\big)x_G^*U_{GA}^* + \big((T + S) - 2P\big)x_G^*\bigg) =& -\mu_{U_{GA}^*, 0} + \mu_{U_{GA}^*,1} \\
    2\bigg(-2\big((R + P) - (T + S)\big)x_G^*U_{gA}^* - \big((T + S) - 2P\big)x_G^* + 2\big((R + P) - (T + S)\big)U_{gA}^* + \big((T + S) - 2P\big) \bigg) =& -\mu_{U_{gA}^*, 0} + \mu_{U_{gA}^*,1}
\end{align}

Complementary slackness conditions are

\begin{align}
    -\mu_{x_G^*,0}x_G^* =& 0 \\
    \mu_{x_G^*,1}\big(x_G^* - 1\big) =& 0 \\
    -\mu_{U_{GA}^*,0}U_{GA}^* =& 0 \\
    \mu_{U_{GA}^*,1}\big(U_{GA}^* - 1\big) =& 0
    -\mu_{U_{gA}^*,0}U_{gA}^* =& 0 \\
    \mu_{U_{gA}^*,1}\big(U_{gA}^* - 1\big) =& 0
\end{align}

where $\mu_0, \mu_1$ are Lagrange multipliers.

### Case 1: interior equilibrium $(0 < x_G^* < 1, 0 < U_{GA}^* < 1, 0 < U_{gA}^* < 1)$

In an interior equilibrium, complementary slackness conditions imply that all Lagrange multipliers are zero (i.e., $\mu_{x_G^*, 0} =\mu_{x_G^*, 1} =\mu_{U_{GA}^*, 0} =\mu_{U_{GA}^*, 0} =\mu_{U_{gA}^*, 0} =\mu_{U_{gA}^*, 0}=0$).

Our first order conditions reduce to the following.

\begin{align}
    2\bigg(\big((R + P) - (T + S)\big)U_{GA}^{*2} + \big((T + S) - 2P\big)U_{GA}^* + P - \big((R + P) - (T + S)\big)U_{gA}^{*2} - \big((T + S) - 2P\big)U_{gA}^*\bigg) =& 0 \\
    2\bigg(2\big((R + P) - (T + S)\big)x_G^*U_{GA}^* + \big((T + S) - 2P\big)x_G^*\bigg) =& 0 \\
    2\bigg(-2\big((R + P) - (T + S)\big)x_G^*U_{gA}^* - \big((T + S) - 2P\big)x_G^* + 2\big((R + P) - (T + S)\big)U_{gA}^* + \big((T + S) - 2P\big) \bigg) =& 0
\end{align}

Rearranging the second first-order condition yields an expression for the optimal value of $U_{GA}^*$.

$$ \bar{U}_{GA}^* = \frac{1}{2}\left(\frac{2P - (T + S)}{(R + P) - (T + S)}\right) $$

Substituting this result into the first first-order condition and rearranging yields an identical exprssion for the optimal value of $U_{gA}^*$.

$$ \bar{U}_{gA}^* = \frac{1}{2}\left(\frac{2P - (T + S)}{(R + P) - (T + S)}\right) $$

Substituting this result into the third first-order condition yields a result which implies that the optimal value for $x_G^*$ is indeterminate (i.e., the objective is flat when holding $U_{GA}^*$ and $U_{gA}^*$ fixed). 


In [40]:
first_order_conditions = sym.Matrix([equilibrium_total_offspring.diff(xG_star, 1),
                                     equilibrium_total_offspring.diff(UGA_star, 1),
                                     equilibrium_total_offspring.diff(UgA_star, 1)])

In [45]:
optimal_UGA_star, = sym.solve(first_order_conditions[1,0], UGA_star)

In [48]:
optimal_UgA_star, = sym.solve(first_order_conditions[0,0].subs({UGA_star: optimal_UGA_star}), UgA_star)

In [52]:
# optimal value for xG_star is indeterminate!
sym.simplify(first_order_conditions[2,0].subs({UGA_star: optimal_UGA_star, UgA_star: optimal_UgA_star}))

0

In [53]:
jacobian = first_order_conditions.jacobian([xG_star, UGA_star, UgA_star])
simplified_jacobian = sym.simplify(jacobian.subs({UGA_star: optimal_UGA_star, UgA_star: optimal_UgA_star}))
e1, e2, e3 = (simplified_jacobian.eigenvals()
                                 .keys())

In [60]:
e1

-4*P*xG_star + 4*P - 4*R*xG_star + 4*R + 4*S*xG_star - 4*S + 4*T*xG_star - 4*T

In [61]:
e2

4*P*xG_star + 4*R*xG_star - 4*S*xG_star - 4*T*xG_star

Requirement for total offspring to optimal a local maximum at the above values derived above is for the Hessian to be negative semi-definite. This requirement will be satisfied if and only if 

$$ R + P < T + S. $$

### Case 2: equilibrium with $\bar{x}_G^*=1$, $(0 < U_{GA}^* < 1, 0 < U_{gA}^* < 1)$

In this equilibrium, complementary slackness conditions imply that all Lagrange multipliers are zero (i.e., $\mu_{x_G^*, 0} =\mu_{U_{GA}^*, 0} =\mu_{U_{GA}^*, 0} =\mu_{U_{gA}^*, 0} =\mu_{U_{gA}^*, 0}=0$) except $\mu_{x_G^*, 1}  > 0$.

Our first order conditions reduce to the following.

\begin{align}
    2\bigg(\big((R + P) - (T + S)\big)U_{GA}^{*2} + \big((T + S) - 2P\big)U_{GA}^* + P - \big((R + P) - (T + S)\big)U_{gA}^{*2} - \big((T + S) - 2P\big)U_{gA}^*\bigg) =& \mu_{x_G^*, 1} \\
    2\bigg(2\big((R + P) - (T + S)\big)U_{GA}^* + \big((T + S) - 2P\big)\bigg) =& 0 \\
    2\bigg(-2\big((R + P) - (T + S)\big)U_{gA}^* - \big((T + S) - 2P\big) + 2\big((R + P) - (T + S)\big)U_{gA}^* + \big((T + S) - 2P\big) \bigg) =& 0
\end{align}

Rearranging the second first-order condition yields an expression for the optimal value of $U_{GA}^*$.

$$ \bar{U}_{GA}^* = \frac{1}{2}\left(\frac{2P - (T + S)}{(R + P) - (T + S)}\right) $$

Substituting this optimal value of $U_{GA}^*$ into the first first-order condition and rearranging we find that the inequality will hold so long as 

$$ \big((R + P) - (T + S)\big)\big(U_{gA}^* - \bar{U}_{GA}^*\big)^2 + P > 0 $$

which requires $R + P > T + S$. Finally, rearranging the third first-order condition implies that the optimal value for $U_{gA}^*$ is indeterminate: so long as $R + P > T + S$, then $\bar{x}_G^*=1$ for any value of $U_{gA}^*$.